## Connect to a calculation node for analysis

In [1]:
srun -p short --pty bash -i

(base) 

: 1

In [2]:
hostname

hostname
node20.alineos.net
(base) 

: 1

## Prepare directory for the analysis

In [20]:
mkdir ~/Trainings_Bacterial_genomics

echo $?
0
(base) 

: 1

In [3]:
cd ~/Trainings_Bacterial_genomics

cd ~/Trainings_Bacterial_genomics
(base) 

: 1

In [38]:
rm -rf ~/Trainings_Bacterial_genomics/*

rm -rf ~/Trainings_Bacterial_genomics/*
<rf /data2/formation/Bacterial_genomics/* ~/Training                        s_Bacterial_genomics
(base) 

: 1

## Genome Assembly (from Oxford Nanopore Technologies (ONT) long reads) (using Flye)

In [4]:
module load bioinfo/flye/2.9

module load bioinfo/flye/2.9
(base) 

: 1

In [39]:
mkdir assembly

mkdir assembly
(base) 

: 1

In [7]:
flye --nano-raw /data2/formation/Bacterial_genomics/raw_data/CIX4108.sample.fastq -o assembly >>unicycler.log 2>&1

: 127

In [41]:
grep -c ">" assembly/assembly.fasta

grep -c ">" assembly/assembly.fasta
3
(base) 

: 1

## Separate chromosomal and plasmid scaffolds (using MOB-Suite)

In [29]:
module load system/singularity/3.6.0

module load system/singularity/3.6.0
(base) 

: 1

In [30]:
singularity exec /data2/formation/Bacterial_genomics/tools/mob_suite_3.0.3.sif mob_recon -i assembly/assembly.fasta -o assembly_seb.mob_recon >>mob_recon.log 2>&1

<y_seb.fasta -o assembly_seb.mob_recon >>mob_recon.l                        og 2>&1recon >>mob_recon.
(base) 

: 1

In [ ]:
ls assembly.mob_recon

In [44]:
more assembly.mob_recon/contig_report.txt

more assembly.mob_recon/contig_report.txt
sample_id	molecule_type	primary_cluster_id	secondary_cluster_id	contig_id	size	gc	md5	circularity_status	rep_type(s)	rep_type_accession(s)	relaxase_type(s)	relaxase_type_accession(s)	mpf_type	mpf_type_accession(s)	orit_type(s)	orit_accession(s)	predicted_mobility	mash_nearest_neighbor	mash_neighbor_distance	mash_neighbor_identification	repetitive_dna_id	repetitive_dna_type	filtering_reason
assembly	chromosome	-	-	1	4641611	63.66705439124476	864c3f2a7dcba2f9974176ad1fe41fa2	not tested	-	-	MOBP	NC_007507_00032	-	-	-	-	-	-	-	-	-	-	-
assembly	plasmid	novel_6b16544b04f85272e8ae0e02880d4665	-	2	125366	61.611601231593895	a17133681ebbdd0a98e04cf0bb33a048	not tested	rep_cluster_1289	000607__CP000620_00033	MOBP	CP009886_00027	-	-	-	-	-	CP033195	0.0766828	Xanthomonas oryzae pv. oryzae	-	-	-
assembly	plasmid	novel_6b16544b04f85272e8ae0e02880d4665	-	3	78253	61.84427434091984	8d19e4b2158e0f49e7cc9531ec069cf5	not tested	-	-	-	-	-	-	-	-	-	CP033195	0.0766828	Xa

: 1

## Genome annotation (using Prokka)

In [ ]:
module load bioinfo/prokka/1.13.3

In [37]:
prokka assembly_seb.mob_recon/chromosome.fasta --prefix assembly --force --outdir prokka_out >>prokka.log 2>&1

<assembly --force --outdir prokka_out >>prokka.log 2                        >&1okka_out >>prokka.log 
(base) 

: 1

In [49]:
head -40 prokka_out/assembly.gbf

head -40 prokka_out/PROKKA*.gbf
LOCUS       1                    4641611 bp    DNA     linear       31-MAY-2022
DEFINITION  Genus species strain strain.
ACCESSION   
VERSION
KEYWORDS    .
SOURCE      Genus species
  ORGANISM  Genus species
            Unclassified.
COMMENT     Annotated using prokka 1.13.3 from
            https://github.com/tseemann/prokka.
FEATURES             Location/Qualifiers
     source          1..4641611
                     /organism="Genus species"
                     /mol_type="genomic DNA"
                     /strain="strain"
     CDS             complement(156..548)
                     /locus_tag="DGCPDMLJ_00001"
                     /inference="ab initio prediction:Prodigal:2.6"
                     /codon_start=1
                     /transl_table=11
                     /product="hypothetical protein"
                     /translation="MARWASHRSRATSRLINRAAIKQYSQSTPVSCSAGAGGTRWPCI
                     RICCVRGGISAIVIGMAVVHGMPHGMVHLLHIGWLWCRWLHVQAMPLHA

: 1

In [50]:
head -10 prokka_out/assembly.gff

head -10 prokka_out/PROKKA*.gff
##gff-version 3
##sequence-region 1 1 4641611
1	Prodigal:2.6	CDS	156	548	.	-	0	ID=DGCPDMLJ_00001;inference=ab initio prediction:Prodigal:2.6;locus_tag=DGCPDMLJ_00001;product=hypothetical protein
1	Prodigal:2.6	CDS	1191	2228	.	+	0	ID=DGCPDMLJ_00002;Name=btuB_1;gene=btuB_1;inference=ab initio prediction:Prodigal:2.6,protein motif:HAMAP:MF_01531;locus_tag=DGCPDMLJ_00002;product=Vitamin B12 transporter BtuB
1	Prodigal:2.6	CDS	2341	2550	.	+	0	ID=DGCPDMLJ_00003;inference=ab initio prediction:Prodigal:2.6;locus_tag=DGCPDMLJ_00003;product=hypothetical protein
1	Prodigal:2.6	CDS	3011	3571	.	+	0	ID=DGCPDMLJ_00004;inference=ab initio prediction:Prodigal:2.6;locus_tag=DGCPDMLJ_00004;product=hypothetical protein
1	Prodigal:2.6	CDS	3944	4351	.	+	0	ID=DGCPDMLJ_00005;inference=ab initio prediction:Prodigal:2.6;locus_tag=DGCPDMLJ_00005;product=hypothetical protein
1	Prodigal:2.6	CDS	4359	4616	.	+	0	ID=DGCPDMLJ_00006;inference=ab initio prediction:Prodigal:2.6;locus_tag=D

: 1

In [51]:
grep COG prokka_out/assembly.gff | head -5

grep COG prokka_out/PROKKA*.gff | head -5
1	Prodigal:2.6	CDS	28224	28628	.	+	0	ID=DGCPDMLJ_00053;Name=tap_1;db_xref=COG:COG0840;gene=tap_1;inference=ab initio prediction:Prodigal:2.6,similar to AA sequence:UniProtKB:P07018;locus_tag=DGCPDMLJ_00053;product=Methyl-accepting chemotaxis protein IV
1	Prodigal:2.6	CDS	49807	50037	.	+	0	ID=DGCPDMLJ_00095;Name=cheW;db_xref=COG:COG0835;gene=cheW;inference=ab initio prediction:Prodigal:2.6,similar to AA sequence:UniProtKB:P0A964;locus_tag=DGCPDMLJ_00095;product=Chemotaxis protein CheW
1	Prodigal:2.6	CDS	60391	60714	.	-	0	ID=DGCPDMLJ_00112;eC_number=4.2.1.3;Name=acnB_1;db_xref=COG:COG1049;gene=acnB_1;inference=ab initio prediction:Prodigal:2.6,similar to AA sequence:UniProtKB:Q8ZRS8;locus_tag=DGCPDMLJ_00112;product=Aconitate hydratase B
1	Prodigal:2.6	CDS	61051	61416	.	-	0	ID=DGCPDMLJ_00114;eC_number=4.2.1.3;Name=acnB_2;db_xref=COG:COG1049;gene=acnB_2;inference=ab initio prediction:Prodigal:2.6,similar to AA sequence:UniProtKB:Q8ZRS8;locus_tag=DG

: 1

In [4]:
module load system/python/2.7.10
python /data2/formation/Bacterial_genomics/tools/SkewIT/src/gcskew.py -i assembly/assembly.fasta -o gcskew.txt -k 5000

module load system/python/2.7.10
<src/gcskew.py -i assembly/assembly.fasta -o gcskew.                        txtsembly.fasta -o gcskew
PROGRAM START TIME: 05-31-2022 11:56:14
    Input file: assembly/assembly.fasta
    Output: gcskew.txt
    Window size (bp): 20000
    Frequency (bp): 20000
>> Reading sequence file assembly/assembly.fasta
	3 seqs found
>> Calculating/Saving GC Skew for 3 seqs
	Finished calculating gc skew for 3 sequences
PROGRAM FINISH TIME: 05-31-2022 11:56:15
(base) 

: 1

In [5]:
head -5 gcskew.txt

head -5 gcskew.txt
Sequence	Index	GC Skew (20kb)
1	0	-0.04238693
1	20000	-0.01993487
1	40000	-0.04095728
1	60000	-0.00981323
(base) 

: 1

In [13]:
grep -v 'Sequence' gcskew.txt | awk {'print "Chr "$2+1" "$2+5000" "$3'}  >gcskew.circos.txt

<| awk {'print "Chr "$2+1" "$2+20000" "$3'}  >gcskew                        .circos.txt "$3'}  >gcske
(base) 

: 1

In [24]:
awk {'if ($3 == "CDS" && $7 == "+")print "Chr "$4" "$5'} prokka_out/PROKKA*.gff >genes_plus.txt
awk {'if ($3 == "CDS" && $7 == "-")print "Chr "$4" "$5'} prokka_out/PROKKA*.gff >genes_minus.txt

< "+")print "Chr "$4" "$5'} prokka_out/PROKKA*.gff >                        genes_plus.txtROKKA*.gff 
< "-")print "Chr "$4" "$5'} prokka_out/PROKKA*.gff >                        genes_minus.txtOKKA*.gff 
(base) 

: 1

## Visualize genome annotation (using Circos)

In [13]:
module load bioinfo/circos/0.69-6

module load bioinfo/circos/0.69-6
(base) 

: 1

In [12]:
echo "chr - Chr 1 0 4600000 black" >karyotype.txt

In [ ]:
cp -rf /data2/formation/Bacterial_genomics/minimal_circos.conf .

In [26]:
circos -conf minimal_circos.conf

circos -conf minimal_circos.conf
debuggroup summary 0.38s welcome to circos v0.69-6 31 July 2017 on Perl 5.024000
debuggroup summary 0.38s current working directory /home/adereeper/Trainings_Bacterial_genomics
debuggroup summary 0.38s command /usr/local/circos-0.69-6/bin/circos -conf minimal_circos.conf
debuggroup summary 0.38s loading configuration from file minimal_circos.conf
Use of uninitialized value in subroutine entry at /usr/local/circos-0.69-6/bin/../lib/Circos/Configuration.pm line 781.
debuggroup summary 0.38s found conf file minimal_circos.conf
debuggroup summary 0.60s debug will appear for these features: output,summary
debuggroup summary 0.60s bitmap output image ./circos.png
debuggroup summary 0.60s SVG output image ./circos.svg
debuggroup summary 0.60s parsing karyotype and organizing ideograms
debuggroup summary 0.60s karyotype has 1 chromosomes of total size 4,600,001
debuggroup summary 0.60s applying global and local scaling
debuggroup summary 0.60s allocating image,

: 1

In [27]:
cp -rf circos.svg circos1.svg

cp -rf circos.svg circos1.svg
(base) 

: 1

<img src="Trainings_Bacterial_genomics/circos1.svg" align="center" width="70%" style="display: block; margin: auto;"/> 

## Retrieve public genomes available for comparison

Public genomes can be retrived using NCBI : https://www.ncbi.nlm.nih.gov/datasets/genomes/

In [27]:
wget https://ftp.ncbi.nlm.nih.gov/genomes/GENOME_REPORTS/prokaryotes.txt

< https://ftp.ncbi.nlm.nih.gov/genomes/GENOME_REPORT                        S/prokaryotes.txt
--2022-05-31 14:28:11--  https://ftp.ncbi.nlm.nih.gov/genomes/GENOME_REPORTS/prokaryotes.txt
Résolution de ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)... 130.14.250.11, 130.14.250.12, 2607:f220:41f:250::230, ...
Connexion vers ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)|130.14.250.11|:443...connecté.
requête HTTP transmise, en attente de la réponse...200 OK
Longueur: 130463851 (124M) [text/plain]
Sauvegarde en : «prokaryotes.txt»

100%[======================================>] 130 463 851 22,0MB/s   ds 11s    

2022-05-31 14:28:24 (11,5 MB/s) - «prokaryotes.txt» sauvegardé [130463851/130463851]

(base) 

: 1

In [28]:
grep "Xanthomonas oryzae pv. oryzae" prokaryotes.txt | grep -c 'chromosome:'

< "Xanthomonas oryzae pv. oryzae" prokaryotes.txt |                         grep -c 'chromosome:'
95
(base) 

: 1

In [29]:
grep "Xanthomonas oryzae pv. oryzicola" prokaryotes.txt | grep -c 'chromosome:'

< "Xanthomonas oryzae pv. oryzicola" prokaryotes.txt                         | grep -c 'chromosome:'
17
(base) 

: 1

In [2]:
grep "Xanthomonas oryzae pv. oryzicola" prokaryotes.txt | grep 'chromosome:' | head -4

Xanthomonas oryzae pv. oryzicola	129394	PRJNA248159	248159	Proteobacteria	Gammaproteobacteria	4.43083	64.1	chromosome:NZ_CP007810.1/CP007810.1	-	1	4137	3486	2015/06/08	2021/11/23	Complete Genome	Key Laboratory of Agricultural Biodiversity for Plant Disease Management under the Ministry of Education	SAMN02793996	GCA_001021915.1	REPR	ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/001/021/915/GCA_001021915.1_ASM102191v1	-	YM15
Xanthomonas oryzae pv. oryzicola	129394	PRJNA237971	237971	Proteobacteria	Gammaproteobacteria	5.0801	64	chromosome:NZ_CP007221.1/CP007221.1	-	1	4690	3924	2015/03/04	2022/01/21	Complete Genome	Cornell University	SAMN02640212	GCA_000940825.1	-	ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/000/940/825/GCA_000940825.1_ASM94082v1	27148456	CFBP7342
Xanthomonas oryzae pv. oryzicola	129394	PRJNA280380	280380	Proteobacteria	Gammaproteobacteria	5.01777	63.9	chromosome:NZ_CP011959.1/CP011959.1	-	1	4564	3815	2015/06/30	2021/11/23	Complete Genome	Bogdanove Lab, Plant Pathology and Plan

In [5]:
grep "Xanthomonas oryzae pv. oryzae" prokaryotes.txt | grep 'chromosome:' | head -4

Xanthomonas oryzae pv. oryzae	64187	PRJNA485465	485465	Proteobacteria	Gammaproteobacteria	4.99067	63.7	chromosome:NZ_CP031697.1/CP031697.1	-	1	4647	3652	2019/02/04	2022/03/27	Complete Genome	Martin Luther University Halle-Wittenberg	SAMN09791860	GCA_004136375.1	-	ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/004/136/375/GCA_004136375.1_ASM413637v1	-	ICMP3125
Xanthomonas oryzae pv. oryzae	64187	PRJNA269487	269487	Proteobacteria	Gammaproteobacteria	5.09305	63.7	chromosome:NZ_CP040687.1/CP040687.1	-	1	4757	3787	2019/05/28	2022/03/20	Complete Genome	Bacterial Genomics and Evolution Laboratory, CSIR-Institute of Microbial Technology, Sector 39-A, Chandigarh, India	SAMN03252586	GCA_001929235.2	-	ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/001/929/235/GCA_001929235.2_ASM192923v2	-	IXO1088
Xanthomonas oryzae pv. oryzae	64187	PRJNA497605	497605	Proteobacteria	Gammaproteobacteria	5.05039	63.7	chromosome:NZ_CP033192.1/CP033192.1	-	1	4713	3695	2019/03/19	2022/04/07	Complete Genome	University of Florid

In [20]:
wget https://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/004/136/375/GCA_004136375.1_ASM413637v1/GCA_004136375.1_ASM413637v1_genomic.fna.gz
wget https://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/001/929/235/GCA_001929235.2_ASM192923v2/GCA_001929235.2_ASM192923v2_genomic.fna.gz
wget https://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/001/021/915/GCA_001021915.1_ASM102191v1/GCA_001021915.1_ASM102191v1_genomic.fna.gz
wget https://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/000/940/825/GCA_000940825.1_ASM94082v1/GCA_000940825.1_ASM94082v1_genomic.fna.gz

<6/375/GCA_004136375.1_ASM413637v1/GCA_004136375.1_A                        SM413637v1_genomic.fna.gz
--2022-06-01 08:55:23--  https://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/004/136/375/GCA_004136375.1_ASM413637v1/GCA_004136375.1_ASM413637v1_genomic.fna.gz
Résolution de ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)... 130.14.250.12, 130.14.250.11, 2607:f220:41f:250::229, ...
Connexion vers ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)|130.14.250.12|:443...connecté.
requête HTTP transmise, en attente de la réponse...200 OK
Longueur: 1415838 (1,3M) [application/x-gzip]
Sauvegarde en : «GCA_004136375.1_ASM413637v1_genomic.fna.gz»

100%[======================================>] 1 415 838   1,94MB/s   ds 0,7s   

2022-06-01 08:55:25 (1,94 MB/s) - «GCA_004136375.1_ASM413637v1_genomic.fna.gz» sauvegardé [1415838/1415838]

<9/235/GCA_001929235.2_ASM192923v2/GCA_001929235.2_A                        SM192923v2_genomic.fna.gz
--2022-06-01 08:55:25--  https://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/001/929/

: 1

In [21]:
gunzip *_genomic.fna.gz

gunzip *_genomic.fna.gz
(base) 

: 1

In [25]:
module load bioinfo/prokka/1.13.3
prokka GCA_004136375.1_ASM413637v1_genomic.fna --prefix GCA_004136375.1_ASM413637v1 --outdir prokka_out --force >>prokka.log 2>&1
prokka GCA_001929235.2_ASM192923v2_genomic.fna --prefix GCA_001929235.2_ASM192923v2 --outdir prokka_out --force >>prokka.log 2>&1
prokka GCA_001021915.1_ASM102191v1_genomic.fna --prefix GCA_001021915.1_ASM102191v1 --outdir prokka_out --force >>prokka.log 2>&1
prokka GCA_000940825.1_ASM94082v1_genomic.fna --prefix GCA_000940825.1_ASM94082v1 --outdir prokka_out --force >>prokka.log 2>&1

module load bioinfo/prokka/1.13.3
<GCA_004136375.1_ASM413637v1 --outdir prokka_out --f                        orce >>prokka.log 2>&1 --
<GCA_001929235.2_ASM192923v2 --outdir prokka_out --f                        orce >>prokka.log 2>&1 --
<GCA_001021915.1_ASM102191v1 --outdir prokka_out --f                        orce >>prokka.log 2>&1 --
<CA_000940825.1_ASM94082v1 --outdir prokka_out --for                        ce >>prokka.log 2>&1 --fo
(base) 

: 1

In [11]:
wget https://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/004/136/375/GCA_004136375.1_ASM413637v1/GCA_004136375.1_ASM413637v1_genomic.gbff.gz
wget https://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/001/929/235/GCA_001929235.2_ASM192923v2/GCA_001929235.2_ASM192923v2_genomic.gbff.gz
wget https://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/004/355/825/GCA_004355825.1_ASM435582v1/GCA_004355825.1_ASM435582v1_genomic.gbff.gz
wget https://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/001/746/615/GCA_001746615.1_ASM174661v1/GCA_001746615.1_ASM174661v1_genomic.gbff.gz
wget https://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/001/021/915/GCA_001021915.1_ASM102191v1/GCA_001021915.1_ASM102191v1_genomic.gbff.gz
wget https://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/000/940/825/GCA_000940825.1_ASM94082v1/GCA_000940825.1_ASM94082v1_genomic.gbff.gz
wget https://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/001/042/835/GCA_001042835.1_ASM104283v1/GCA_001042835.1_ASM104283v1_genomic.gbff.gz
wget https://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/001/042/815/GCA_001042815.1_ASM104281v1/GCA_001042815.1_ASM104281v1_genomic.gbff.gz

<CA_004136375.1_ASM413637v1/GCA_004136375.1_ASM41363                        7v1_genomic.gbff.gzSM4136
--2022-06-01 07:05:40--  https://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/004/136/375/GCA_004136375.1_ASM413637v1/GCA_004136375.1_ASM413637v1_genomic.gbff.gz
Résolution de ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)... 130.14.250.10, 165.112.9.230, 2607:f220:41e:250::11, ...
Connexion vers ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)|130.14.250.10|:443...connecté.
requête HTTP transmise, en attente de la réponse...200 OK
Longueur: 3276284 (3,1M) [application/x-gzip]
Sauvegarde en : «GCA_004136375.1_ASM413637v1_genomic.gbff.gz.1»

100%[======================================>] 3 276 284   3,89MB/s   ds 0,8s   

2022-06-01 07:05:42 (3,89 MB/s) - «GCA_004136375.1_ASM413637v1_genomic.gbff.gz.1» sauvegardé [3276284/3276284]

<CA_001929235.2_ASM192923v2/GCA_001929235.2_ASM19292                        3v2_genomic.gbff.gzSM1929
--2022-06-01 07:05:42--  https://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/00

: 1

## Convert GenBank annotation files to GFF format

In [15]:
mkdir annotations

In [26]:
cp -rf prokka_out/*.gff annotations

cp -rf prokka_out/*.gff annotations
(base) 

: 1

In [13]:
bp_genbank2gff3.pl annotations/GCA_000940825.1_ASM94082v1_genomic.gbff.gz
bp_genbank2gff3.pl annotations/GCA_001021915.1_ASM102191v1_genomic.gbff.gz
bp_genbank2gff3.pl annotations/GCA_001042815.1_ASM104281v1_genomic.gbff.gz
bp_genbank2gff3.pl annotations/GCA_001042835.1_ASM104283v1_genomic.gbff.gz
bp_genbank2gff3.pl annotations/GCA_001746615.1_ASM174661v1_genomic.gbff.gz
bp_genbank2gff3.pl annotations/GCA_001929235.2_ASM192923v2_genomic.gbff.gz
bp_genbank2gff3.pl annotations/GCA_004136375.1_ASM413637v1_genomic.gbff.gz
bp_genbank2gff3.pl annotations/GCA_004355825.1_ASM435582v1_genomic.gbff.gz

<gff3.pl annotations/GCA_000940825.1_ASM94082v1_geno                        mic.gbff.gz
Possible precedence issue with control flow operator at /usr/local/perl-5.24.0/bin/bp_genbank2gff3.pl line 690.
# Input: annotations/GCA_000940825.1_ASM94082v1_genomic.gbff.gz
# working on region:CP007221, Xanthomonas oryzae pv. oryzicola, Xanthomonas oryzae pv. oryzicola strain CFBP7342, complete genome., 16-NOV-2015
# GFF3 saved to ./GCA_000940825.1_ASM94082v1_genomic.gbff.gz.gff
<gff3.pl annotations/GCA_001021915.1_ASM102191v1_gen                        omic.gbff.gz
Possible precedence issue with control flow operator at /usr/local/perl-5.24.0/bin/bp_genbank2gff3.pl line 690.
# Input: annotations/GCA_001021915.1_ASM102191v1_genomic.gbff.gz
# working on region:CP007810, Xanthomonas oryzae pv. oryzicola, Xanthomonas oryzae pv. oryzicola strain YM15, complete genome., 08-JUN-2015
# GFF3 saved to ./GCA_001021915.1_ASM102191v1_genomic.gbff.gz.gff
<gff3.pl annotations/GCA_001042815.1_ASM104281v1_gen   

: 1

In [16]:
mv *gff annotations

mv *gff annotations


: 1

## Comparison between 2 genomes (with Minimap2 and circos)

In [ ]:
module load bioinfo/R/3.6.1

In [21]:
module load bioinfo/minimap2/2.24
module load bioinfo/circos/0.69-6

module load bioinfo/minimap2/2.24
(base) module load bioinfo/circos/0.69-6
(base) 

: 1

#### Compare 2 oryzae

In [22]:
minimap2 GCA_004136375.1_ASM413637v1_genomic.fna GCA_001929235.2_ASM192923v2_genomic.fna -o minimap2.paf

<3637v1_genomic.fna GCA_001929235.2_ASM192923v2_geno                        mic.fna -o minimap2.pafen
[M::mm_idx_gen::0.411*0.52] collected minimizers
[M::mm_idx_gen::0.449*0.73] sorted minimizers
[M::main::0.449*0.73] loaded/built the index for 1 target sequence(s)
[M::mm_mapopt_update::0.466*0.74] mid_occ = 75
[M::mm_idx_stat] kmer size: 15; skip: 10; is_hpc: 0; #seq: 1
[M::mm_idx_stat::0.478*0.74] distinct minimizers: 764850 (94.10% are singletons); average occurrences: 1.218; average spacing: 5.356; total length: 4990672
[M::worker_pipeline::25.200*0.99] mapped 1 sequences
[M::main] Version: 2.24-r1122
[M::main] CMD: minimap2 -o minimap2.paf GCA_004136375.1_ASM413637v1_genomic.fna GCA_001929235.2_ASM192923v2_genomic.fna
[M::main] Real time: 25.226 sec; CPU: 25.027 sec; Peak RSS: 0.424 GB
(base) 

: 1

In [24]:
awk {'if (($4-$3)>100000 && $5=="+")print $1" "$3" "$4" "$6" "$8" "$9'} minimap2.paf >links.txt
awk {'if (($4-$3)>100000 && $5=="-")print $1" "$3" "$4" "$6" "$9" "$8'} minimap2.paf >links_inverted.txt

<=="+")print $1" "$3" "$4" "$6" "$8" "$9'} minimap2.                        paf >links.txt'} minimap2
<=="-")print $1" "$3" "$4" "$6" "$9" "$8'} minimap2.                        paf >links_inverted.txtp2
(base) 

: 1

In [15]:
cp -rf minimal_circos.conf link_circos.conf

cp -rf minimal_circos.conf link_circos.conf
(base) 

: 1

Edit the Circos configuration file to add informations about links
```
<links>

<link>
file          = /home/adereeper/Trainings_Bacterial_genomics/links.txt
stroke_color     = vdgrey
stroke_thickness = 2
color            = grey_a3
radius        = 0.95r
bezier_radius = 0.1r
thickness     = 2
ribbon        = yes
</link>

<link>
file          = /home/adereeper/Trainings_Bacterial_genomics/links_inverted.txt
stroke_color     = vdblue
stroke_thickness = 2
color            = blue_a3
radius        = 0.95r
bezier_radius = 0.1r
thickness     = 2
ribbon        = yes
</link>

</links>
```

Edit a karyotype file with the two genomes and their size

Make one of the genome in the opposite way by entering this line in the Circos configuration file

chromosomes_reverse = CP031697.1

In [28]:
circos -conf link_circos.conf

circos -conf link_circos.conf
debuggroup summary 0.22s welcome to circos v0.69-6 31 July 2017 on Perl 5.024000
debuggroup summary 0.22s current working directory /home/adereeper/Trainings_Bacterial_genomics
debuggroup summary 0.22s command /usr/local/circos-0.69-6/bin/circos -conf link_circos.conf
debuggroup summary 0.22s loading configuration from file link_circos.conf
Use of uninitialized value in subroutine entry at /usr/local/circos-0.69-6/bin/../lib/Circos/Configuration.pm line 781.
debuggroup summary 0.22s found conf file link_circos.conf
debuggroup summary 0.40s debug will appear for these features: output,summary
debuggroup summary 0.40s bitmap output image ./circos.png
debuggroup summary 0.40s SVG output image ./circos.svg
debuggroup summary 0.40s parsing karyotype and organizing ideograms
debuggroup summary 0.41s karyotype has 2 chromosomes of total size 10,083,726
debuggroup summary 0.41s applying global and local scaling
debuggroup summary 0.41s allocating image, colors and

: 1

In [29]:
cp -rf circos.svg circos2.svg

cp -rf circos.svg circos2.svg
(base) 

: 1

<img src="Trainings_Bacterial_genomics/circos2.svg" align="center" width="70%" style="display: block; margin: auto;"/>

#### Compare oryzae and oryzicola

In [33]:
minimap2 GCA_000940825.1_ASM94082v1_genomic.fna GCA_001929235.2_ASM192923v2_genomic.fna -o minimap2.paf

<082v1_genomic.fna GCA_001929235.2_ASM192923v2_genom                        ic.fna -o minimap2.pafeno
[M::mm_idx_gen::0.225*0.85] collected minimizers
[M::mm_idx_gen::0.273*1.21] sorted minimizers
[M::main::0.273*1.21] loaded/built the index for 1 target sequence(s)
[M::mm_mapopt_update::0.288*1.20] mid_occ = 74
[M::mm_idx_stat] kmer size: 15; skip: 10; is_hpc: 0; #seq: 1
[M::mm_idx_stat::0.299*1.19] distinct minimizers: 801992 (93.39% are singletons); average occurrences: 1.182; average spacing: 5.360; total length: 5080102
[M::worker_pipeline::10.114*1.01] mapped 1 sequences
[M::main] Version: 2.24-r1122
[M::main] CMD: minimap2 -o minimap2.paf GCA_000940825.1_ASM94082v1_genomic.fna GCA_001929235.2_ASM192923v2_genomic.fna
[M::main] Real time: 10.145 sec; CPU: 10.203 sec; Peak RSS: 0.225 GB
(base) 

: 1

In [34]:
awk {'if (($4-$3)>100000 && $5=="+")print $1" "$3" "$4" "$6" "$8" "$9'} minimap2.paf >links.txt
awk {'if (($4-$3)>100000 && $5=="-")print $1" "$3" "$4" "$6" "$9" "$8'} minimap2.paf >links_inverted.txt

<=="+")print $1" "$3" "$4" "$6" "$8" "$9'} minimap2.                        paf >links.txt'} minimap2
<=="-")print $1" "$3" "$4" "$6" "$9" "$8'} minimap2.                        paf >links_inverted.txtp2
(base) 

: 1

In [35]:
circos -conf link_circos.conf

circos -conf link_circos.conf
debuggroup summary 0.22s welcome to circos v0.69-6 31 July 2017 on Perl 5.024000
debuggroup summary 0.22s current working directory /home/adereeper/Trainings_Bacterial_genomics
debuggroup summary 0.22s command /usr/local/circos-0.69-6/bin/circos -conf link_circos.conf
debuggroup summary 0.22s loading configuration from file link_circos.conf
Use of uninitialized value in subroutine entry at /usr/local/circos-0.69-6/bin/../lib/Circos/Configuration.pm line 781.
debuggroup summary 0.22s found conf file link_circos.conf
debuggroup summary 0.39s debug will appear for these features: output,summary
debuggroup summary 0.39s bitmap output image ./circos.png
debuggroup summary 0.39s SVG output image ./circos.svg
debuggroup summary 0.40s parsing karyotype and organizing ideograms
debuggroup summary 0.40s karyotype has 2 chromosomes of total size 10,173,156
debuggroup summary 0.40s applying global and local scaling
debuggroup summary 0.40s allocating image, colors and

: 1

In [36]:
cp -rf circos.svg circos3.svg

cp -rf circos.svg circos3.svg
(base) 

: 1

<img src="Trainings_Bacterial_genomics/circos3.svg" align="center" width="70%" style="display: block; margin: auto;"/>

## Compare genome similarity using Average Nucleotide Identity (ANI) (fastANI)

In [4]:
module load bioinfo/fastani/1.33

In [7]:
ls GCA_00*fna >list_genomes.txt

In [8]:
fastANI --rl list_genomes.txt --ql list_genomes.txt -o fastani.out --matrix

>>>>>>>>>>>>>>>>>>
Reference = [GCA_000940825.1_ASM94082v1_genomic.fna, GCA_001021915.1_ASM102191v1_genomic.fna, GCA_001929235.2_ASM192923v2_genomic.fna, GCA_004136375.1_ASM413637v1_genomic.fna]
Query = [GCA_000940825.1_ASM94082v1_genomic.fna, GCA_001021915.1_ASM102191v1_genomic.fna, GCA_001929235.2_ASM192923v2_genomic.fna, GCA_004136375.1_ASM413637v1_genomic.fna]
Kmer size = 16
Fragment length = 3000
Threads = 1
ANI output file = fastani.out
>>>>>>>>>>>>>>>>>>
INFO [thread 0], skch::main, Count of threads executing parallel_for : 1
INFO [thread 0], skch::Sketch::build, window size for minimizer sampling  = 24
INFO [thread 0], skch::Sketch::build, minimizers picked from reference = 1572160
INFO [thread 0], skch::Sketch::index, unique minimizers = 504251
INFO [thread 0], skch::Sketch::computeFreqHist, Frequency histogram of minimizers = (1, 75036) ... (898, 1)
INFO [thread 0], skch::Sketch::computeFreqHist, consider all minimizers during lookup.
INFO [thread 0], skch::main, Time spent s

##### Check that genomes are closer within the same pathovar than between pathovars.

##### Which are the closest genomes?

In [2]:

#import matplotlib.pyplot as plt
import numpy as np

a = np.random.random((16, 16))
plt.imshow(a, cmap='hot', interpolation='nearest')
plt.show()

ModuleNotFoundError: No module named 'numpy'

## Gene content comparison / Pangenome analysis (using Roary)

In [2]:
module load bioinfo/roary/3.12.0

In [5]:
rm -rf roary_out
roary -f roary_out -e -n annotations/*gff

Use of uninitialized value in require at /usr/local/perl-5.24.0/lib/perl5/x86_64-linux-thread-multi/Encode.pm line 69.

Please cite Roary if you use any of the results it produces:
    Andrew J. Page, Carla A. Cummins, Martin Hunt, Vanessa K. Wong, Sandra Reuter, Matthew T. G. Holden, Maria Fookes, Daniel Falush, Jacqueline A. Keane, Julian Parkhill,
	"Roary: Rapid large-scale prokaryote pan genome analysis", Bioinformatics, 2015 Nov 15;31(22):3691-3693
    doi: http://doi.org/10.1093/bioinformatics/btv421
	Pubmed: 26198102

Use of uninitialized value in require at /usr/local/perl-5.24.0/lib/perl5/x86_64-linux-thread-multi/Encode.pm line 69.
Use of uninitialized value in require at (eval 711) line 1.
Academic tradition requires you to cite works you base your article on.
When using programs that use GNU Parallel to process data for publication
please cite:

  O. Tange (2011): GNU Parallel - The Command-Line Power Tool,
  ;login: The USENIX Magazine, February 2011:42-47.

This helps fun

In [4]:
head -10 roary_out/gene_presence_absence.csv

head -10 roary_out/gene_presence_absence.csv
"Gene","Non-unique Gene name","Annotation","No. isolates","No. sequences","Avg sequences per isolate","Genome Fragment","Order within Fragment","Accessory Fragment","Accessory Order with Fragment","QC","Min group size nuc","Max group size nuc","Avg group size nuc","GCA_000940825.1_ASM94082v1_genomic.gbff.gz","GCA_001021915.1_ASM102191v1_genomic.gbff.gz","GCA_001042815.1_ASM104281v1_genomic.gbff.gz","GCA_001042835.1_ASM104283v1_genomic.gbff.gz","GCA_001746615.1_ASM174661v1_genomic.gbff.gz","GCA_001929235.2_ASM192923v2_genomic.gbff.gz","GCA_004136375.1_ASM413637v1_genomic.gbff.gz","GCA_004355825.1_ASM435582v1_genomic.gbff.gz","PROKKA_05312022"
"rpoC","","DNA-directed RNA polymerase subunit beta'","9","9","1","1","13165","","","","383","4217","3791","BE73_19305.p01","FE36_04050.p01","ACU16_17350.p01","ACU17_17205.p01","ATY43_13975.p01","10bd397a0f88aca11976f4a4f6631125_14380","4e95a45ca635958849417c3d368d746d_13974","cc532a0030968c6887cea9dec5a

: 1

In [26]:
more roary_out/summary_statistics.txt

Core genes	(99% <= strains <= 100%)	0
Soft core genes	(95% <= strains < 99%)	0
Shell genes	(15% <= strains < 95%)	8167
Cloud genes	(0% <= strains < 15%)	0
Total genes	(0% <= strains <= 100%)	8167


In [ ]:
module purge
module load system/python/3.7.2
python /usr/local/roary-3.12.0/contrib/roary_plots/roary_plots.py roary_out/accessory_binary_genes.fa.newick roary_out/gene_presence_absence.csv

<img src="Trainings_Bacterial_genomics/pangenome_pie.png" align="center" width="70%" style="display: block; margin: auto;"/> 

<img src="Trainings_Bacterial_genomics/pangenome_matrix.png"/>

## Association tests with metadata (Pan-GWAS)

In [3]:
module load bioinfo/scoary/1.6.16

module load bioinfo/scoary/1.6.16
(base) 

: 1

In [4]:
cp -rf /data2/formation/Bacterial_genomics/pangenome_100xantho ./
cd pangenome_100xantho

<rf /data2/formation/Bacterial_genomics/pangenome_10                        0xantho ./
(base) cd pangenome_100xantho
(base) 

: 1

In [5]:
scoary -t traits.txt -g gene_presence_absence.csv

scoary -t traits.txt -g gene_presence_absence.csv
==== Scoary started ====
Command: /usr/local/miniconda3/envs/scoary-1.6.16/bin/scoary -t traits.txt -g gene_presence_absence.csv
Reading gene presence absence file
Creating Hamming distance matrix based on gene presence/absence
Building UPGMA tree from distance matrix
Reading traits file
ERROR: Some isolates in your gene presence absence file were not represented in your traits file. These will count as MISSING data and will not be included.
Finished loading files into memory.


==== Performing statistics ====
-- Filtration options --
Individual (Naive):    0.05
Collapse genes:    False


Tallying genes and performing statistical analyses
Gene-wise counting and Fisher's exact tests for trait: pathovar
100.00%
Adding p-values adjusted for testing multiple hypotheses

Storing results: pathovar
Calculating max number of contrasting pairs for each nominally significant gene
100.00%
Storing results to file


==== Finished ====
Checked a tota

: 1

#### How many clusters are specifically present in pathovar oryzae (absent in oryzicola)

In [7]:
awk -F "\",\"" {'if ($4==0 && $5==75)print $_'} pathovar_*.results.csv | wc -l

<-F "\",\"" {'if ($4==0 && $5 ==75)print $_'} pathov                        ar_*.results.csv | wc -l
28
(base) 

: 1

#### How many clusters are specifically present in pathovar oryzicola (absent in oryzae)

In [9]:
awk -F "\",\"" {'if ($4==15 && $5==0)print $_'} pathovar_*.results.csv | wc -l

<-F "\",\"" {'if ($4==15 && $5 ==0)print $_'} pathov                        ar_*.results.csv | wc -l
110
(base) 

: 1